In [ ]:
!pip install numba
!pip install ipywidgets

%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.ticker as tick
import numpy as np 
from numba import jit
import math 
from itertools import product

from ipywidgets import interact, interactive, fixed




In [60]:
# molifireの積分値を計算するための関数


#y = math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def moli(x,a,b,c):
    return math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def int_moli(c):
    delta_t = 0.00001
    x = np.arange(0.000001,1,delta_t)

    S = 0
    y = moli(x,1,1,c)
    for i,ele in enumerate(y):
        if i != 0:
            S = S + delta_t * ele
    return S    


In [91]:
# ここは動作原理の説明用のセルです。ツールを使うだけの場合は飛ばしてください。
# moli(x,a,b,c)がどんな形か描画してみる。

print("moliの形を描画してみる")
@interact(a=(0.1, 3.0, 0.01), b=(0.01, 3, 0.01), c = (0.01, 10.0, 0.01)  )
def moli_view(a=1.0, b=1, c=1):
    # 曲率グラフの描画
    delta_t = 0.005
    t = np.arange(0.00001, b, delta_t)
    kappa =  moli(t,a,b,c)
    
    plt.subplot(1, 2, 1)
    plt.gca().set_aspect('equal')
    plt.plot(t,kappa)
    plt.title("kappa plot")
    plt.grid()
    plt.xlim(0, 3)
    plt.ylim(0, 3)

    # 軌跡のx座標, y座標の描画
    c_traj_x = np.zeros(len(t))
    c_traj_y = np.zeros(len(t))

    I_kappa = np.zeros(len(kappa))
    for i,ele in enumerate(kappa):
        if i != 0:
            I_kappa[i] = I_kappa[i-1] + delta_t * ele 
    
    for i,ele in enumerate(I_kappa):
        if i != 0:
            c_traj_x[i] = c_traj_x[i-1] + delta_t *  math.cos(ele)
            c_traj_y[i] = c_traj_y[i-1] + delta_t *  math.sin(ele)
    
    plt.subplot(1, 2, 2)
    plt.gca().set_aspect('equal')
    plt.scatter(c_traj_x, c_traj_y, marker = '.',s=10,lw = 0)
    plt.title("trajectory plot")
    plt.grid()
    plt.xlim(-1, 2)
    plt.ylim(-1, 3)
    
    plt.tight_layout() 
    plt.show()
    
    
    print("曲率の曲線下面積 = 曲線終点での角度[rad] = " + str(int_moli(c) *b/a))

moliの形を描画してみる


interactive(children=(FloatSlider(value=1.0, description='a', max=3.0, min=0.1, step=0.01), FloatSlider(value=…

In [92]:

# 軌跡を作るためのクラス
class CalcTraj:
    shape_factor = 0
    path_length = 0
    target_ang = 0
    delta_t = 0
    tread = 0
    
    s = None
    kappa =  None
    D_kappa = None
    I_kappa = None
    c_traj_x = None
    c_traj_y = None
    l_traj_x = None
    l_traj_y = None
    r_traj_x = None
    r_traj_y = None
    
    # 軌跡のパラメータをセットする
    def set_param(self, shape_factor_, path_length_, target_ang_, delta_t_, tread_):
        self.shape_factor = shape_factor_
        self.path_length = path_length_
        self.target_ang = target_ang_
        self.delta_t = delta_t_
        self.tread = tread_
        
    # 軌跡を計算する    
    @jit
    def calc_traj(self):
        self.s = np.arange(0.00001, self.path_length, self.delta_t)
        
        # a * b * S = target_ang
        c = self.shape_factor
        b = self.path_length
        S = int_moli(c)
        a = 1/(self.target_ang/b/S)
        
        # 曲率
        self.kappa = math.e / a * np.exp( (-1)/(1- np.abs(2*self.s/b -1)**c))
        
        # 曲率の微分
        self.D_kappa = np.gradient(self.kappa)/ self.delta_t
        
        # 曲率の積分
        self.I_kappa = np.zeros(len(self.kappa))
        for i,ele in enumerate(self.kappa):
            if i != 0:
                self.I_kappa[i] = self.I_kappa[i-1] + self.delta_t * ele 

        # 軌跡のx座標, y座標を求める        
        self.c_traj_x = np.zeros(len(self.kappa))
        self.c_traj_y = np.zeros(len(self.kappa))

        self.l_traj_x = np.zeros(len(self.kappa))
        self.l_traj_y = np.zeros(len(self.kappa))

        self.r_traj_x = np.zeros(len(self.kappa))
        self.r_traj_y = np.zeros(len(self.kappa))
        
        for i,ele in enumerate(self.I_kappa):
            self.r_traj_x[0] =  0.0
            self.r_traj_x[0] = - self.tread/2

            self.l_traj_x[0] = 0.0
            self.l_traj_x[0] = +self.tread/2

            if i != 0:
                self.c_traj_x[i] = self.c_traj_x[i-1] + self.delta_t *  math.cos(ele)
                self.c_traj_y[i] = self.c_traj_y[i-1] + self.delta_t *  math.sin(ele)
                self.r_traj_x[i] = self.c_traj_x[i] + math.sin(ele)*self.tread/2
                self.r_traj_y[i] = self.c_traj_y[i] - math.cos(ele)*self.tread/2

                self.l_traj_x[i] = self.c_traj_x[i] - math.sin(ele)*self.tread/2
                self.l_traj_y[i] = self.c_traj_y[i] + math.cos(ele)*self.tread/2
        
        
        
        
    
    # 軌跡が終わったポイントのx座標
    def get_end_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1]
    # 軌跡が終わったポイントのy座標    
    def get_end_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1]

    # 速度一定で走ったときの時間と角速度のリストを返す
    def get_omega(self, v):
        return self.s /v, self.kappa * v
    
    # 速度一定で走ったときの時間と角速度(degree)のリストを返す
    def get_omega_degree(self, v):
        t, rad = self.get_omega(v) 
        return t, rad * 180 /math.pi
    
    # 速度一定で走ったときの時間と角加速度のリストを返す
    def get_alpha(self, v):
        t, rad = self.get_omega(v)
        alpha = np.gradient(rad)/ (self.delta_t/v)
        return t,alpha 

    # 速度一定で走ったときの時間と角加速度(degree)のリストを返す
    def get_alpha_degree(self, v):
        t, alpha = self.get_alpha(v)
        return t, alpha * 180/math.pi
        
    # 速度一定で走ったときに右タイヤにかかる力を返す
    def get_r_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f = np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
    
    # 速度一定で走ったときに左タイヤにかかる力を返す
    def get_l_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f =  np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
        

In [117]:
from pylab import rcParams
rcParams['figure.figsize'] = 10,10

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
shape_factor__ = 2.6
path_length__ = 0.09
target_ang__ = 90.0
v__ = 0.5

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),target_ang=(0.01,360.0,0.1) ,  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, target_ang =target_ang__, v=v__):
    traj = CalcTraj()
    traj.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , delta_t_=delta_t, tread_=tread)
    traj.calc_traj()
    
    t, omega = traj.get_omega_degree(v)
    t, alpha = traj.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj.get_r_force(v, m, I)

    plt.subplot(3, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(3, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(3, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()
    plt.tight_layout() 
    plt.show()

    plt.axes().set_aspect('equal')
    plt.scatter(traj.c_traj_x, traj.c_traj_y, marker = '.',s=10,lw = 0)
    plt.scatter(traj.r_traj_x, traj.r_traj_y, marker = '.',s=10,lw = 0)
    plt.scatter(traj.l_traj_x, traj.l_traj_y, marker = '.',s=10,lw = 0)

    #plt.plot([0,0],[0,0.18*2])
    plt.axvline(0, ls='-.')       # x=** の線
    plt.axvline(0.09, ls='-.')       # x=** の線
    plt.axvline(0.18, ls='-.')       # x=** の線

    #plt.axvline(0.18, ls='-.')       # x=** の線

    plt.axhline(0, ls='-.')       # x=** の線
    plt.axhline(0.09, ls='-.')       # x=** の線
    plt.axhline(-0.09, ls='-.')       # x=** の線

    #plt.axhline(0.18, ls='-.')       # x=** の線
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.27, 0.27, -0.27, 0.27])

    plt.grid(True)
    plt.show()

      
    print("end x[m] " + str(traj.get_end_x()))
    print("end y[m] " + str(traj.get_end_y()))
    print("len(kappa) " + str(len(traj.kappa)))
    print("s resolution[m] " + str(traj.delta_t))
    
    # kappaを書き出す。分解能はお好みで
    #for i,(s, kappa) in enumerate(zip(traj.s, traj.kappa)):
    #    if i % 10 == 0:
    #        print(str(kappa)+ "," )
    
    


interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…